In [1]:
#####Note:run in python2######
# same initialization of weights because same random seed used

#SGD has

import math
import numpy as np
import sys
from PIL import Image
import matplotlib.pyplot as plt 

#for shuffling data
from sklearn.utils import shuffle
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

np.random.seed(42)

In [2]:
#########
#MLP

# initialisation of the weights he_normal
def weights(noHiddenLayers,sizeOfLayers):

    W=[]
    b=[]

    for i in range(0,noHiddenLayers+1):
        
        W.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+sizeOfLayers[i])),
                                   (sizeOfLayers[i+1],sizeOfLayers[i])) )
        b.append( np.random.normal(1e-4,np.sqrt(2*1.0/(sizeOfLayers[i+1]+1)),
                                   (sizeOfLayers[i+1],1)) )

    W=np.array(W)
    b=np.array(b)
    
    return W,b


#mlp forward pass
#layer
def layer(w,x,b):
    out = np.matmul(w,x)+b
    return out

def apply_activationMLP(Activation_function,inp):
    
    #activation functions
    if Activation_function == 'relu':
        return np.where(inp<0,0,inp)
    elif Activation_function == "tanh":
        return np.tanh(inp)
    elif Activation_function == "sigmoid":
        return 1.0/(1+np.exp(-1.0*inp))
    elif Activation_function == "softmax":
        return (1.0/(np.sum(np.exp(inp),axis=0)))*(np.exp(inp))

#forward path
def forward_path(noHiddenLayers,X,W,b,Actfnvect):

    out=[]
    
    z=apply_activationMLP(Actfnvect[0],np.array(layer(W[0],X,b[0])))
    out.append(np.array(z))

    for i in range(1,noHiddenLayers):
        z=apply_activationMLP(Actfnvect[i],np.array(layer(W[i],out[i-1],b[i])))
        out.append(np.array(z))

    if noHiddenLayers > 0:
        z=apply_activationMLP(Actfnvect[-1],np.array(layer(W[-1],out[-1],b[-1])))
        out.append(np.array(z))

    y_pred = out[-1]

    return np.array(out),np.array(y_pred)


In [3]:
#only to import data
from keras import backend as K
import keras
from keras.datasets import mnist


# #import data
iris_data = load_iris() # load the iris dataset

x = iris_data.data
y_ = iris_data.target.reshape(-1, 1) # Convert data to a single column

# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
# Split the data for training and testing
x_train , x_test, y_train , y_test = train_test_split(x, y, test_size=0.20)

print(x_train.shape)
print(y_train.shape)

Using TensorFlow backend.


(120, 4)
(120, 3)


In [4]:
# run MLP algorithm

x_train = np.moveaxis(x_train,0,-1)
y_train = np.moveaxis(y_train,0,-1)
x_test = np.moveaxis(x_test,0,-1)
y_test = np.moveaxis(y_test,0,-1)

print x_train.shape
print y_train.shape
print x_test.shape
print y_test.shape

(4, 120)
(3, 120)
(4, 30)
(3, 30)


In [5]:
#derivative of relu
def der_relu(x):
    return np.where(x == 0,0,1)

# backpropation
def backprop(y,y_true,z,T,u,M,x):
    
    dy = (y-y_true)
    
    # layer 3
    dT  = np.matmul(dy,z.T)
    db3 = np.sum(dy,axis=1).reshape(-1,1)
    
    # layer 2
    s   = np.matmul(T.T,dy)
    s   = s*der_relu(z)
    dM  = np.matmul(s,u.T)
    db2 = np.sum(s,axis=1).reshape(-1,1)
    
    # layer 1
    sm  = np.matmul(M.T,s)
    sm  = sm*der_relu(u)
    dW  = np.matmul(sm,x.T)
    db1 = np.sum(sm,axis=1).reshape(-1,1)
    
    return dW,dM,dT,db1,db2,db3

In [6]:
###
################################
# Training parameters
num_classes = 3
epochs = 600
learning_rate=0.0001
batch_size = 10
alpha = 0.9
################################

#MLP PARAMETERS
noHiddenLayers=2

#also includes the input vector dimension and output vector dimension
sizeOfLayers=[x_train.shape[0],10,10,num_classes]

sizeofOutput=num_classes

Actfnvect = ['relu','relu','softmax']

In [7]:
W,b = weights(noHiddenLayers,sizeOfLayers)

In [8]:
# (a)
# Momentum descent

# updating the weights


v = {"W[0]" : np.zeros(W[0].shape) , "W[1]" : np.zeros(W[1].shape) ,"W[2]" : np.zeros(W[2].shape),"b[0]": np.zeros(b[0].shape)
     ,"b[1]" :np.zeros(b[1].shape) , "b[2]" : np.zeros(b[2].shape)}

for i in range(epochs):
    loss=0
    for j in np.arange(0,x_train.shape[1],batch_size):       
        dW = np.zeros(W[0].shape)
        dM = np.zeros(W[1].shape)
        dT = np.zeros(W[2].shape)
        db1 = np.zeros(b[0].shape)
        db2 = np.zeros(b[1].shape)
        db3 = np.zeros(b[2].shape)
        
        #Compute interim update
        W[0] = W[0] + alpha*v["W[0]"]
        W[1] = W[1] + alpha*v["W[1]"]
        W[2] = W[2] + alpha*v["W[2]"]
        b[0] = b[0] + alpha*v["b[0]"]
        b[1] = b[1] + alpha*v["b[1]"]
        b[2] = b[2] + alpha*v["b[2]"]
        
        for k in range(0,batch_size):
            # forward pass
            x = x_train[:,j+k].reshape(-1,1)
            y = y_train[:,j+k].reshape(-1,1)
            out,y_pred=forward_path(noHiddenLayers,x,W,b,Actfnvect)
            # backpropagation
            dWtemp,dMtemp,dTtemp,db1temp,db2temp,db3temp=backprop(y_pred,y,out[1],W[-1],out[0],W[1],x)
            
            dW=dW+dWtemp
            db1=db1+db1temp
            
            dM=dM+dMtemp
            db2=db2+db2temp
            
            dT=dT+dTtemp
            db3=db3+db3temp
        
            # calculate the loss
            loss = loss + (-1.0*np.sum(y*np.log(y_pred)))
            
        # Updating the weights using Nesterov MOMENTUM Approach
        
        dW=dW*(1.0/batch_size)
        dM=dM*(1.0/batch_size)
        dT=dT*(1.0/batch_size)
        db1=db1*(1.0/batch_size)
        db2=db2*(1.0/batch_size)
        db3=db3*(1.0/batch_size)
    
        #Compute velocity update
        v["W[0]"] = alpha*v["W[0]"] - learning_rate*(dW)
        v["W[1]"] = alpha*v["W[1]"] - learning_rate*(dM)
        v["W[2]"] = alpha*v["W[2]"] - learning_rate*(dT)
        v["b[0]"] = alpha*v["b[0]"] - learning_rate*(db1)
        v["b[1]"] = alpha*v["b[1]"] - learning_rate*(db2)
        v["b[2]"] = alpha*v["b[2]"] - learning_rate*(db3)
        
        #Apply update
        W[0] = W[0] + v["W[0]"]
        W[1] = W[1] + v["W[1]"]
        W[2] = W[2] + v["W[2]"]
        b[0] = b[0] + v["b[0]"]
        b[1] = b[1] + v["b[1]"]
        b[2] = b[2] + v["b[2]"]
        
    
    #print the loss in each epoch
    print('Epoch:'+str(i)+'         Loss:'+str(loss))

Epoch:0         Loss:254.1806143958583
Epoch:1         Loss:225.60720200334669
Epoch:2         Loss:199.1778017341242
Epoch:3         Loss:180.99145385678588
Epoch:4         Loss:168.60863538208898
Epoch:5         Loss:159.43546697158013
Epoch:6         Loss:152.32405815518612
Epoch:7         Loss:146.70684061898706
Epoch:8         Loss:141.56895226362371
Epoch:9         Loss:136.35049181781608
Epoch:10         Loss:131.8917995004891
Epoch:11         Loss:127.91283473370167
Epoch:12         Loss:124.07604829384536
Epoch:13         Loss:120.24037258727454
Epoch:14         Loss:116.20574004561178
Epoch:15         Loss:111.05236627869064
Epoch:16         Loss:105.23640602480168
Epoch:17         Loss:100.03327486605606
Epoch:18         Loss:96.0908893238185
Epoch:19         Loss:92.97600313926928
Epoch:20         Loss:90.26667663391463
Epoch:21         Loss:87.87737643916007
Epoch:22         Loss:85.73456170370008
Epoch:23         Loss:83.7621546165463
Epoch:24         Loss:81.936580731446

Epoch:214         Loss:18.343230433424065
Epoch:215         Loss:18.26090228682715
Epoch:216         Loss:18.179255344920332
Epoch:217         Loss:18.098427481761878
Epoch:218         Loss:18.017967893726087
Epoch:219         Loss:17.939147137497905
Epoch:220         Loss:17.860963166132017
Epoch:221         Loss:17.78353930271416
Epoch:222         Loss:17.706954782223477
Epoch:223         Loss:17.63080522456825
Epoch:224         Loss:17.556054701437727
Epoch:225         Loss:17.482029546071473
Epoch:226         Loss:17.408237238986693
Epoch:227         Loss:17.335791207528192
Epoch:228         Loss:17.264074332903167
Epoch:229         Loss:17.192168767231717
Epoch:230         Loss:17.12278540582002
Epoch:231         Loss:17.0541390601464
Epoch:232         Loss:16.98539066734905
Epoch:233         Loss:16.917756178583844
Epoch:234         Loss:16.851279043029137
Epoch:235         Loss:16.784764098926196
Epoch:236         Loss:16.719074382552076
Epoch:237         Loss:16.654132375941032

Epoch:426         Loss:10.81906097068854
Epoch:427         Loss:10.805347879790807
Epoch:428         Loss:10.791718664120998
Epoch:429         Loss:10.778172612191957
Epoch:430         Loss:10.76470901490458
Epoch:431         Loss:10.751327174201592
Epoch:432         Loss:10.738026401535748
Epoch:433         Loss:10.724806015062907
Epoch:434         Loss:10.711665339505537
Epoch:435         Loss:10.698603706819096
Epoch:436         Loss:10.685628267598359
Epoch:437         Loss:10.672925733182028
Epoch:438         Loss:10.660196538980644
Epoch:439         Loss:10.6474487064079
Epoch:440         Loss:10.634793507311167
Epoch:441         Loss:10.622243373400172
Epoch:442         Loss:10.609767456099993
Epoch:443         Loss:10.597356222404423
Epoch:444         Loss:10.585016074534973
Epoch:445         Loss:10.572750093638673
Epoch:446         Loss:10.560556321973268
Epoch:447         Loss:10.548432915209006
Epoch:448         Loss:10.536379479747891
Epoch:449         Loss:10.524395837942

In [9]:
out,y_pred=forward_path(noHiddenLayers,x_test,W,b,Actfnvect)

In [10]:
def predict(y):
    return np.argmax(y)

yvect=[]
y_trurevect=[]

for i in range(0,x_test.shape[1]):
    yvect.append(predict(y_pred[:,i]))
    y_trurevect.append(predict(y_test[:,i]))

# find accuracy
from sklearn.metrics import accuracy_score
#predicting test accuracy
print(accuracy_score(y_trurevect, yvect))

1.0


In [11]:
# to see the output vs true values

print y_trurevect
print yvect

[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
[1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 2, 0, 0, 0, 0, 1, 2, 1, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 0, 0]
